In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from string import punctuation


In [2]:
#Display the dataset using Pandas

df1 = pd.read_csv("Data/Eclipse/EP_dup.csv", sep=";")
df2 = pd.read_csv("Data/Eclipse/EP_nondup.csv", sep=";")
df3 = pd.read_csv("Data/Mozilla/M_Duplicate BRs.csv", sep=";")  
df4 = pd.read_csv("Data/Mozilla/M_NonDuplicate BRs.csv", sep=";")
df5 = pd.read_csv("Data/ThunderBird/dup_TB.csv", sep=";")
df6 = pd.read_csv("Data/ThunderBird/Nondup_TB.csv", sep=";")

frames=[df1,df2,df3,df4,df5,df6]
data=pd.concat(frames)
data

,Issue_id,Duplicated_issue,Title1,Description1,Title2,Description2,Label
0,25,28126,cvs ui need vcm prefs default repo connection gc,it would be helpful if there was a notion of d...,wizards patch standard public cvs repositories,this patch adds a convenient way to check thin...,1
1,40,20,need connect to team stream gcqpkw,i would like to be able to connect to a team s...,workspace files,thought it would be useful if the set of repo ...,1
2,48,22,make sure can future store other project refer...,project references come in three flavours . p...,persist sharing recommendations and project ve...,project descriptions dont store sharing recomm...,1
3,61,60,.vcmmeta showing as change gdqtgw,useruser install drop into declipse user ...,need custom .vcmignore comparemerge gdqt,useruser install drop into declipse user ...,1
4,94,2,repositories view all file types open to the t...,when browsing files in the repositories view i...,opening repository resources doesnt honor type...,opening repository resource open the default ...,1
...,...,...,...,...,...,...,...
9900,418747,953052,thunderbird outlook migrate lost outlooks pst ...,i installed thunderbird hoping i could impor...,i am using thunderbird esr latest but i keep ...,i am using thunderbird .. esr latest but i...,0
9901,418821,953165,urls in mail with comma opens with c in browser,when clicking in text mail not html on url w...,picture in feed not showing,. i linked the rssfeed from httpwww.nu.nlf...,0
9902,418832,953249,greek attachments not displayed correctly,i am using gmail imap accounts. when i recei...,add the flagcolpng to linux,httpmxr.mozilla.orgcommcentralsearchstringflag...,0
9903,419035,953375,branch only random crash when i want to displ...,the problem appears with icedove the debian ...,replace undefined gpromptservice with services...,patch v discovered a call to undefined gprom...,0


In [3]:
data = data.reset_index(inplace = False)
data

,index,Issue_id,Duplicated_issue,Title1,Description1,Title2,Description2,Label
0,0,25,28126,cvs ui need vcm prefs default repo connection gc,it would be helpful if there was a notion of d...,wizards patch standard public cvs repositories,this patch adds a convenient way to check thin...,1
1,1,40,20,need connect to team stream gcqpkw,i would like to be able to connect to a team s...,workspace files,thought it would be useful if the set of repo ...,1
2,2,48,22,make sure can future store other project refer...,project references come in three flavours . p...,persist sharing recommendations and project ve...,project descriptions dont store sharing recomm...,1
3,3,61,60,.vcmmeta showing as change gdqtgw,useruser install drop into declipse user ...,need custom .vcmignore comparemerge gdqt,useruser install drop into declipse user ...,1
4,4,94,2,repositories view all file types open to the t...,when browsing files in the repositories view i...,opening repository resources doesnt honor type...,opening repository resource open the default ...,1
...,...,...,...,...,...,...,...,...
122070,9900,418747,953052,thunderbird outlook migrate lost outlooks pst ...,i installed thunderbird hoping i could impor...,i am using thunderbird esr latest but i keep ...,i am using thunderbird .. esr latest but i...,0
122071,9901,418821,953165,urls in mail with comma opens with c in browser,when clicking in text mail not html on url w...,picture in feed not showing,. i linked the rssfeed from httpwww.nu.nlf...,0
122072,9902,418832,953249,greek attachments not displayed correctly,i am using gmail imap accounts. when i recei...,add the flagcolpng to linux,httpmxr.mozilla.orgcommcentralsearchstringflag...,0
122073,9903,419035,953375,branch only random crash when i want to displ...,the problem appears with icedove the debian ...,replace undefined gpromptservice with services...,patch v discovered a call to undefined gprom...,0


In [4]:
#Summary of the DataFrame
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122075 entries, 0 to 122074
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   index             122075 non-null  int64 
 1   Issue_id          122075 non-null  int64 
 2   Duplicated_issue  122075 non-null  int64 
 3   Title1            122075 non-null  object
 4   Description1      122075 non-null  object
 5   Title2            122075 non-null  object
 6   Description2      122075 non-null  object
 7   Label             122075 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 7.5+ MB


In [5]:
# Checking the presence of Null values
data.isnull().sum()

index               0
Issue_id            0
Duplicated_issue    0
Title1              0
Description1        0
Title2              0
Description2        0
Label               0
dtype: int64

In [6]:
# NOT NEEDED

# Replace Null values for columns that will be used
#data["Title1"].fillna( method ='ffill', inplace = True) 
#data["Title2"].fillna( method ='ffill', inplace = True) 
#data["Description1"].fillna( method ='ffill', inplace = True) 
#data["Description2"].fillna( method ='ffill', inplace = True) 
#data["Label"].fillna( method ='ffill', inplace = True) 

#data.isnull().sum()

In [7]:
data

,index,Issue_id,Duplicated_issue,Title1,Description1,Title2,Description2,Label
0,0,25,28126,cvs ui need vcm prefs default repo connection gc,it would be helpful if there was a notion of d...,wizards patch standard public cvs repositories,this patch adds a convenient way to check thin...,1
1,1,40,20,need connect to team stream gcqpkw,i would like to be able to connect to a team s...,workspace files,thought it would be useful if the set of repo ...,1
2,2,48,22,make sure can future store other project refer...,project references come in three flavours . p...,persist sharing recommendations and project ve...,project descriptions dont store sharing recomm...,1
3,3,61,60,.vcmmeta showing as change gdqtgw,useruser install drop into declipse user ...,need custom .vcmignore comparemerge gdqt,useruser install drop into declipse user ...,1
4,4,94,2,repositories view all file types open to the t...,when browsing files in the repositories view i...,opening repository resources doesnt honor type...,opening repository resource open the default ...,1
...,...,...,...,...,...,...,...,...
122070,9900,418747,953052,thunderbird outlook migrate lost outlooks pst ...,i installed thunderbird hoping i could impor...,i am using thunderbird esr latest but i keep ...,i am using thunderbird .. esr latest but i...,0
122071,9901,418821,953165,urls in mail with comma opens with c in browser,when clicking in text mail not html on url w...,picture in feed not showing,. i linked the rssfeed from httpwww.nu.nlf...,0
122072,9902,418832,953249,greek attachments not displayed correctly,i am using gmail imap accounts. when i recei...,add the flagcolpng to linux,httpmxr.mozilla.orgcommcentralsearchstringflag...,0
122073,9903,419035,953375,branch only random crash when i want to displ...,the problem appears with icedove the debian ...,replace undefined gpromptservice with services...,patch v discovered a call to undefined gprom...,0


In [8]:
# Counting the unique values in "label" column
a = data.pivot_table(index = ['Label'], aggfunc ='size') 
a = a.reset_index()
a.columns= ["Values", "Counts"]
a

,Values,Counts
0,0,80960
1,1,41115


In [9]:
data['TitleDescription1'] = data['Title1'].str.cat(data['Description1'],sep=" ")
data['TitleDescription2'] = data['Title2'].str.cat(data['Description2'],sep=" ")

data

,index,Issue_id,Duplicated_issue,Title1,Description1,Title2,Description2,Label,TitleDescription1,TitleDescription2
0,0,25,28126,cvs ui need vcm prefs default repo connection gc,it would be helpful if there was a notion of d...,wizards patch standard public cvs repositories,this patch adds a convenient way to check thin...,1,cvs ui need vcm prefs default repo connection...,wizards patch standard public cvs repositories...
1,1,40,20,need connect to team stream gcqpkw,i would like to be able to connect to a team s...,workspace files,thought it would be useful if the set of repo ...,1,need connect to team stream gcqpkw i would lik...,workspace files thought it would be useful if ...
2,2,48,22,make sure can future store other project refer...,project references come in three flavours . p...,persist sharing recommendations and project ve...,project descriptions dont store sharing recomm...,1,make sure can future store other project refer...,persist sharing recommendations and project ve...
3,3,61,60,.vcmmeta showing as change gdqtgw,useruser install drop into declipse user ...,need custom .vcmignore comparemerge gdqt,useruser install drop into declipse user ...,1,.vcmmeta showing as change gdqtgw useruser i...,need custom .vcmignore comparemerge gdqt useru...
4,4,94,2,repositories view all file types open to the t...,when browsing files in the repositories view i...,opening repository resources doesnt honor type...,opening repository resource open the default ...,1,repositories view all file types open to the t...,opening repository resources doesnt honor type...
...,...,...,...,...,...,...,...,...,...,...
122070,9900,418747,953052,thunderbird outlook migrate lost outlooks pst ...,i installed thunderbird hoping i could impor...,i am using thunderbird esr latest but i keep ...,i am using thunderbird .. esr latest but i...,0,thunderbird outlook migrate lost outlooks pst ...,i am using thunderbird esr latest but i keep ...
122071,9901,418821,953165,urls in mail with comma opens with c in browser,when clicking in text mail not html on url w...,picture in feed not showing,. i linked the rssfeed from httpwww.nu.nlf...,0,urls in mail with comma opens with c in brows...,picture in feed not showing . i linked the...
122072,9902,418832,953249,greek attachments not displayed correctly,i am using gmail imap accounts. when i recei...,add the flagcolpng to linux,httpmxr.mozilla.orgcommcentralsearchstringflag...,0,greek attachments not displayed correctly i ...,add the flagcolpng to linux httpmxr.mozilla.or...
122073,9903,419035,953375,branch only random crash when i want to displ...,the problem appears with icedove the debian ...,replace undefined gpromptservice with services...,patch v discovered a call to undefined gprom...,0,branch only random crash when i want to displ...,replace undefined gpromptservice with services...


In [10]:
stop_words = set(stopwords.words('english'))
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [11]:
#Remove punctuations and stop words from Bug reports
def words(text, remove_stop_words=True, stem_words=False):
    # Remove punctuation from questions
    text = ''.join([c for c in text if c not in punctuation])
    
    # Lowering the words in questions
    text = text.lower()
    
    # Remove stop words from questions
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Return a list of words
    return(text)

In [12]:
def process(bug_list, bugs):
    for bug in bugs:
        bug_list.append(words(bug))
processed_bug1 = []
processed_bug2 = []
process(processed_bug1, data.TitleDescription1)
process(processed_bug2, data.TitleDescription2)

In [13]:
#Preview some of comparisons between bug report pairs after processing
a = 0 
for i in range(a,a+10):
    print(processed_bug1[i])
    print(processed_bug2[i])
    print()

cvs ui need vcm prefs default repo connection gc would helpful notion default repository connection connected project wouldnt walk list everytime sure goes prefs never created maybe create mark one default minimum default repository type cvscc would helpful pulldown repo connection wizard default
wizards patch standard public cvs repositories patch adds convenient way check things commonly used free software cvs repositories discussion platformvcmdev list attached final version patch pending requests make changes

need connect team stream gcqpkw would like able connect team stream directly add content workspace could added new repository wizard currently need least steps populate new workspace team stream add repository open repository browser expand head see pr gcqky itpvcmwinnt able add head workspace select children head add workspace kmpm moving future bbpm least cvs team stream cannot used container projects operation would meaningful cvs repository add workspace frequent operatio

In [14]:
#Define tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer = 'word',
                        stop_words = 'english',
                        lowercase = True,
                        max_features = 300,
                        norm = 'l1')

In [15]:
#Concatenating TitleDescription Columns
words = pd.concat([data.TitleDescription1, data.TitleDescription2], axis = 0)
words.head()

0    cvs ui need vcm prefs  default repo connection...
1    need connect to team stream gcqpkw i would lik...
2    make sure can future store other project refer...
3    .vcmmeta showing as change gdqtgw useruser   i...
4    repositories view all file types open to the t...
dtype: object

In [16]:
#Fitting and transforming bug reports with TFIDF Vectorizer
tfidf.fit(words)
duplicate_1 = tfidf.transform(data.TitleDescription1)
duplicate_2 = tfidf.transform(data.TitleDescription2)

In [17]:
#Assigning independent and dependent variables
x = abs(duplicate_1 - duplicate_2)
y = data['Label']

In [18]:
#Splitting into train and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# LOGISTIC REGRESSION

In [19]:
#Training ML model
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1000,random_state=42)
lr.fit(x_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [20]:
#Predicting the test results
y_pred = lr.predict(x_test) 
y_pred 

array([1, 0, 0, ..., 1, 0, 0], dtype=int64)

In [21]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred) 
TP = cm[1][1] 
TN = cm[0][0]
FP = cm[1][0]
FN = cm[0][1] 
print("True Positive  : ", TP)
print("True Negative  : ", TN)
print("False Positive : ", FP)
print("False Negative : ", FN)

True Positive  :  5547
True Negative  :  15465
False Positive :  2747
False Negative :  656


In [22]:
#Accuracy
Accuracy = (TP + TN) / (TP + TN + FP + FN) 
Accuracy

0.8606184722506656

# NAIVE BAYES 

In [23]:
#Training ML model
from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB()
NB.fit(x_train, y_train)

MultinomialNB()

In [24]:
#Predicting the test results
y_pred = NB.predict(x_test) 
y_pred 

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [25]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred) 
TP = cm[1][1] 
TN = cm[0][0]
FP = cm[1][0]
FN = cm[0][1] 
print("True Positive  : ", TP)
print("True Negative  : ", TN)
print("False Positive : ", FP)
print("False Negative : ", FN)

True Positive  :  0
True Negative  :  16121
False Positive :  8294
False Negative :  0


In [26]:
#Accuracy
Accuracy = (TP + TN) / (TP + TN + FP + FN) 
Accuracy

0.6602908048330944

# SVM 

In [27]:
#Training ML model
from sklearn.linear_model import SGDClassifier
SGD = SGDClassifier(random_state=42)
SGD.fit(x_train, y_train)

SGDClassifier(random_state=42)

In [28]:
#Predicting the test results
y_pred = SGD.predict(x_test) 
y_pred 

array([1, 0, 0, ..., 1, 0, 0], dtype=int64)

In [29]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred) 
TP = cm[1][1] 
TN = cm[0][0]
FP = cm[1][0]
FN = cm[0][1] 
print("True Positive  : ", TP)
print("True Negative  : ", TN)
print("False Positive : ", FP)
print("False Negative : ", FN)

True Positive  :  4422
True Negative  :  15833
False Positive :  3872
False Negative :  288


In [30]:
#Accuracy
Accuracy = (TP + TN) / (TP + TN + FP + FN) 
Accuracy

0.829612942862994

# MLP 

In [31]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred) 
TP = cm[1][1] 
TN = cm[0][0]
FP = cm[1][0]
FN = cm[0][1] 
print("True Positive  : ", TP)
print("True Negative  : ", TN)
print("False Positive : ", FP)
print("False Negative : ", FN)

True Positive  :  4422
True Negative  :  15833
False Positive :  3872
False Negative :  288


In [32]:
#Accuracy
Accuracy = (TP + TN) / (TP + TN + FP + FN) 
Accuracy

0.829612942862994

# CNN

In [55]:
# build model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv1D, GlobalMaxPooling1D


model = Sequential()
model.add(Dense(64,input_shape=(300,)))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 64)                19264     
_________________________________________________________________
dropout_14 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_21 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_15 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_22 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                

In [82]:
results = model.fit(x_train.todense(),y_train,batch_size=64,epochs=5,verbose=1,validation_data=(x_test.todense(),y_test))

Epoch 1/5
1526/1526 [==============================] - 2s 1ms/step - loss: 0.3009 - acc: 0.8831 - val_loss: 0.3243 - val_acc: 0.8702
Epoch 2/5
1526/1526 [==============================] - 2s 1ms/step - loss: 0.2982 - acc: 0.8840 - val_loss: 0.3197 - val_acc: 0.8708
Epoch 3/5
1526/1526 [==============================] - 2s 1ms/step - loss: 0.2974 - acc: 0.8835 - val_loss: 0.3218 - val_acc: 0.8714
Epoch 4/5
1526/1526 [==============================] - 2s 1ms/step - loss: 0.2953 - acc: 0.8840 - val_loss: 0.3209 - val_acc: 0.8716
Epoch 5/5
1526/1526 [==============================] - 2s 1ms/step - loss: 0.2933 - acc: 0.8855 - val_loss: 0.3224 - val_acc: 0.8718


In [83]:
print(np.mean(results.history["val_acc"]))

0.8711529731750488
